In [511]:
import pickle
import folium

import math
import os

In [512]:
with open('graph/final_node_table', 'rb') as f: 
    final_node_table = pickle.load(f)
with open('graph/final_way_table', 'rb') as f:
    final_way_table = pickle.load(f)
with open('graph/final_relation_table', 'rb') as f:
    final_relation_table = pickle.load(f)

In [513]:
road_speeds = {}

In [514]:
# This data structrue will have the final result.
# All of the keys will represent all of the ways that have a bus route go through them.

for key,value in final_way_table.items():
    row = []
    #24 hours * 60 min/hour / 5 min intervals = 288
    for i in range(288):
        row.append(0)
    road_speeds.update({key:row})

In [515]:
# haversine formula implmented
def distance(point1, point2):

    #Radius of earth at latitude 42.89 + elevation of buffalo, ny
    radius = 6368.276 + 0.183

    lat1, lng1 = point1
    lat2, lng2 = point2

    theta1 = math.radians(lat1)
    theta2 = math.radians(lat2)
    phi1 = math.radians(lng1)
    phi2 = math.radians(lng2)

    distance = 2 * radius * math.asin(math.sqrt(math.sin((theta2-theta1)/2)**2 + math.cos(theta1) * math.cos(theta2) * math.sin((phi2-phi1)/2)**2))

    return distance #I'm using kilometers right now

In [516]:
# This function works, but I switched to a different approach because this code was quite slow.
# It finds all nodes and ways within 0.006 degreees of a datapoint.
# However, this is too slow and I opted to only search for nodes and ways given a specific route.
def find_nearest_way(datapoint):

    proximal_nodes = {}
    #Determined a +- value of .006 just by looking at the largest space between nodes
    margin = .006

    for key,value in final_node_table.items():
        if value[0] < datapoint[0] + margin and value[0] > datapoint[0] - margin:
            if value[1] < datapoint[1] + margin and value[1] > datapoint[1] - margin:
                proximal_nodes.update({key:value})
    
    proximal_ways = {}
    for key,value in final_way_table.items():
        for node in value:
            if node in proximal_nodes:
                proximal_ways.update({key:value})
    
    
    
    point1 = datapoint
    min_dist = 9999
    min_way = -1
    min_projection = []
    for key,value in proximal_ways.items():
        for i in range(len(value)-1):
            a = final_node_table[value[i]]
            b = final_node_table[value[i+1]]
            c = point1

            u = [b[0]-a[0], b[1]-a[1]]
            v = [c[0]-a[0], c[1]-a[1]]

            projection = [0,0]
            projection[0] = (u[0]*v[0] + u[1]*v[1]) / (u[0]**2 + u[1]**2) * u[0] + a[0]
            projection[1] = (u[0]*v[0] + u[1]*v[1]) / (u[0]**2 + u[1]**2) * u[1] + a[1]

            if distance(a,c) < min_dist:
                min_dist = distance(a,c)
                min_way = key
                min_projection = a

            if distance(b,c) < min_dist:
                min_dist = distance(b,c)
                min_way = key
                min_projection = b

            if min(a[0],b[0]) <= projection[0] <= max(a[0],b[0]) and min(a[1],b[1]) <= projection[1] <= max(a[1],b[1]):

                if distance(projection, c) < min_dist:
                    min_dist = distance(projection,c)
                    min_way = key
                    min_projection = projection
    
    
    return min_projection, min_way

In [517]:
# A much faster implementation.
# Given the route number, which is provided by the data,
# I can narrow roads to search to those that are in a specific bus route.
# So, this function finds the corresponding relation and searches its
# roads for the nearest one to the datapoint.

def find_nearest_way_in_relations(relation_ids, datapoint):
    
    relations = []
    for id in relation_ids:
        relations.append(final_relation_table[id])
    
    possible_ways = {}
    possible_nodes = {}
    for relation in relations:
        for way in relation[0]:
            if way in final_way_table:
                possible_ways.update({way:final_way_table[way]})
            else:
                possible_nodes.update({way:final_node_table[way]})
    
    for key,value in possible_ways.items():
        for node in value:
            possible_nodes.update({node:final_node_table[node]})
    
    point1 = datapoint
    min_dist = 9999
    min_way = -1
    min_projection = []
    for key,value in possible_ways.items():
        for i in range(len(value)-1):
            a = final_node_table[value[i]]
            b = final_node_table[value[i+1]]
            c = point1

            u = [b[0]-a[0], b[1]-a[1]]
            v = [c[0]-a[0], c[1]-a[1]]

            projection = [0,0]
            projection[0] = (u[0]*v[0] + u[1]*v[1]) / (u[0]**2 + u[1]**2) * u[0] + a[0]
            projection[1] = (u[0]*v[0] + u[1]*v[1]) / (u[0]**2 + u[1]**2) * u[1] + a[1]

            if distance(a,c) < min_dist:
                min_dist = distance(a,c)
                min_way = key
                min_projection = a

            if distance(b,c) < min_dist:
                min_dist = distance(b,c)
                min_way = key
                min_projection = b

            if min(a[0],b[0]) <= projection[0] <= max(a[0],b[0]) and min(a[1],b[1]) <= projection[1] <= max(a[1],b[1]):

                if distance(projection, c) < min_dist:
                    min_dist = distance(projection,c)
                    min_way = key
                    min_projection = projection
    
    
    return min_projection, min_way

In [518]:
# Many roads would get multiple data points.
# The result is that we have to average speed.
# My solution for this was to have an array of speeds and average them in the end.

meta_speeds = {}
for key,value in final_way_table.items():
    row = []
    #24 hours * 60 min/hour / 5 min intervals = 288
    for i in range(288):
        row.append([])
    meta_speeds.update({key:row})

In [519]:
import time
import random

used_ways = set()


directory = 'data/20200801/sorted/'
for filename in os.listdir(directory):
    with open('{}{}'.format(directory,filename)) as f:
        
        
        
        interval = -1
        counter = 0
        points = []
        #m = folium.Map(location=[42.89, -78.82], tiles="OpenStreetMap", zoom_start=12)
        
        # Takes the current data point and the next one and uses the pair for each calculation of distance and speed.
        
        lines = f.readlines()
        for i in range(len(lines)-1):
            line = lines[i][:-1].split(',')
            route_id = int(line[1])
            lat = float(line[7])
            lng = float(line[8])
            total_seconds = int(line[10][11:13]) * 3600 + int(line[10][14:16]) * 60 + int(line[10][17:19])
            interval = math.floor(total_seconds / 300)
            
            line2 = lines[i+1][:-1].split(',')
            route_id2 = int(line2[1])
            lat2 = float(line2[7])
            lng2 = float(line2[8])
            total_seconds2 = int(line2[10][11:13]) * 3600 + int(line2[10][14:16]) * 60 + int(line2[10][17:19])
            interval2 = math.floor(total_seconds2 / 300)
            
            # These are all disqualifying pairs.
            if interval != interval2:
                #print('different interval')
                continue
            if total_seconds == total_seconds2:
                #print('same time')
                continue
            if lat == lat2 and lng == lng2:
                #print('same position')
                continue
            if lat == 99.0 and lng == 999.0:
                #print('bad location 1')
                continue
            if lat2 == 99.0 and lng == 999.0:
                #print('bad location 2')
                continue
            if route_id != route_id:
                #this usually works but it may fail if the bus takes an unofficial road to the new bus route start location
                #so I skip
                continue
            
            possible_relations = set()
            for key,value in final_relation_table.items():
                if str(route_id) in value[1]['ref']:
                    possible_relations.add(key)
            projection, way = find_nearest_way_in_relations(list(possible_relations),[lat,lng])
            
            possible_relations2 = set()
            for key,value in final_relation_table.items():
                if str(route_id2) in value[1]['ref']:
                    possible_relations2.add(key)
            projection2, way2 = find_nearest_way_in_relations(list(possible_relations2), [lat2,lng2])
            
            #print('{}:{}'.format(way,projection))
            #print('{}:{}'.format(way2,projection2))
            #print('{},{}'.format(lat,lng))
            
            speed = distance(projection,projection2) / (total_seconds2 - total_seconds)
            # km/s -> mph
            speed = speed / 1.60934 * 3600
            if speed == 0:
                continue
       
    
    #This code gives the same speed to all ways between the start location and the end location of the bus route.
    #I see very few places where this could fail, but if there is a -<>- looking path the wrong path may be used
    #See route 67, which openstreetmap marks as a backtrack for evidence of this problem
            if way == way2:
                meta_speeds[way][interval].append(speed)
                used_ways.add(way)
                #print('single speed: {}'.format(speed))
            else:
                meta_speeds[way][interval].append(speed)
                meta_speeds[way2][interval].append(speed)
                used_ways.add(way)
                used_ways.add(way2)
                
                '''
                This code took the current bus route, and then found the start and end street locations and then set
                the average speed for each street and all the streets inbetween on the route to the same speed. The
                method worked reasonably but route 67 (and maybe some others I didn't see) has a circular path and
                sometimes the wrong path was taken and their speeds were set incorrectly.
                
                for relation in possible_relations:
                    if way in final_relation_table[relation][0] and way2 in final_relation_table[relation][0]:
                        indices = final_relation_table[relation][0]
                        traversed = False
                        for i in range(len(indices)):

                            if indices[i] == way or indices[i] == way2:
                                traversed = not traversed
                                meta_speeds[indices[i]][interval].append(speed)
                                #print('start or end {} (way {}): {}'.format(traversed,indices[i],speed))
                                used_ways.add(indices[i])
                            elif traversed == True:
                                meta_speeds[indices[i]][interval].append(speed)
                                #print('inbetween (way {}): {}'.format(indices[i],speed))
                                used_ways.add(indices[i])
                        break
                '''
            
            '''
            print(meta_speeds[way])
            print(meta_speeds[way2])
            
            for node in final_way_table[way]:
                folium.Marker(final_node_table[node]).add_to(m)
            folium.Marker(projection).add_to(m)
            folium.Marker([lat,lng],icon=folium.Icon(color='red',icon_color='#FFFF00')).add_to(m)
            
            for node in final_way_table[way2]:
                folium.Marker(final_node_table[node]).add_to(m)
            folium.Marker(projection2).add_to(m)
            folium.Marker([lat2,lng2],icon=folium.Icon(color='red', icon_color='#FFFF00')).add_to(m)
            break
            '''
    print(filename)
#m

1501.csv
1502.csv
1504.csv
1505.csv
1506.csv
1507.csv
1508.csv
1509.csv
1511.csv
1512.csv
1514.csv
1516.csv
1517.csv
1518.csv
1519.csv
1520.csv
1601.csv
1603.csv
1604.csv
1605.csv
1606.csv
1607.csv
1608.csv
1609.csv
1610.csv
1611.csv
1612.csv
1614.csv
1615.csv
1616.csv
1617.csv
1618.csv
1619.csv
1620.csv
1621.csv
1622.csv
1623.csv
1624.csv
1701.csv
1702.csv
1703.csv
1704.csv
1705.csv
1707.csv
1708.csv
1709.csv
1710.csv
1711.csv
1712.csv
1713.csv
1714.csv
1715.csv
1716.csv
1801.csv
1802.csv
1804.csv
1805.csv
1806.csv
1807.csv
1808.csv
1809.csv
1810.csv
1811.csv
1812.csv
1813.csv
1814.csv
1815.csv
1816.csv
1901.csv
1902.csv
1903.csv
1904.csv
1905.csv
1906.csv
1907.csv
1908.csv
1909.csv
1910.csv
1911.csv
1912.csv
1913.csv
1914.csv
1915.csv
1916.csv
1917.csv
1918.csv
1919.csv
1920.csv
1921.csv
1922.csv
2203.csv
2206.csv
2213.csv
2215.csv
2216.csv
2218.csv
2612.csv
2618.csv
2619.csv
2623.csv


In [520]:
for key,value in meta_speeds.items():
    speed_intervals = []
    for i in range(len(value)):
        if len(value[i]) > 1:
            road_speeds[key][i] = sum(value[i])/len(value[i])
        elif len(value[i]) == 1:
            road_speeds[key][i] = value[i][0]

In [521]:
with open('data/20200801_road.csv', 'w') as f:
    line = ""
    line += "Road ID"
    line += ",00:00-00:05"
    
    hour = "00"
    minute = "05"
    while hour != "00" or minute != "00":
        line += "," + hour + ":" + minute + " - "
        minute = str(int(minute)+5).zfill(2)
        if minute == "60":
            minute = "00"
            hour = str(int(hour)+1).zfill(2)
        if hour == "24":
            hour = "00"
        line += hour + ":" + minute
            
    f.write(line)
    f.write('\n')
    
    for key, value in road_speeds.items():
        line = str(key)
        for speed in value:
            line += "," + str(speed)
        f.write(line)
        f.write('\n')

In [522]:
import pickle 
with open('data/20200801_speed_table', 'wb') as f: 
    pickle.dump(road_speeds, f) 